LOG WRITER DIMENSIONS

In [3]:
import pandas as pd
import xlsxwriter
import xlwings as xw

file_format = ['csv', 'avro', 'parquet', 'orc']
schemas = ['st', 'vt', 'pt']
partition = ['horizontal', 'predicate', 'subject']

iterasi = [[[file_format], [schemas], [partition]]]

# ST HORIZONTAL
li =[]
for i in file_format:
    df = pd.read_csv(f'./logs/{i}/100M_st_horizontal.txt',sep = ',', header = None)
    df = df.fillna(0)
    avg = df.mean(axis = 0)
    li.append(avg)

df1 = pd.DataFrame(li, index=[i + "_st_horizontal" for i in file_format])
df1 = df1.transpose()
# # df = df.set_axis(file_format + "_st_horizontal", axis = 1)
df1 = df1.set_axis(["Q"+str(i+1) for i in range(11)], axis = 0)

xlwriter = pd.ExcelWriter('~/Desktop/rank_table.xlsx')
df.to_excel(xlwriter, 'storage_format_100M')
xlwriter.close()

# ST PREDICATE
xls = pd.ExcelFile("~/Desktop/rank_table.xlsx")
df = pd.read_excel(xls, 'storage_format_100M', index_col=0)

li=[]
for i in file_format:
    df = pd.read_csv(f'./logs/{i}/100M_st_predicate.txt',sep = ',', header = None)
    df = df.fillna(0)
    avg = df.mean(axis = 0)
    li.append(avg)

df2 = pd.DataFrame(li, index=[i + "_st_predicate" for i in file_format])
df2 = df2.transpose()         
# # df = df.set_axis(file_format + "_st_horizontal", axis = 1)
df2 = df2.set_axis(["Q"+str(i+1) for i in range(11)], axis = 0)

# CREATE EXCEL
dfs = [df1,df2,df1_ranks,df2_ranks]
xlwriter = pd.ExcelWriter('~/Desktop/rank_table.xlsx', engine='xlsxwriter')
row = 0
for dataframe in dfs:
        dataframe.to_excel(xlwriter, sheet_name = 'storage_format_100M', startrow = row, startcol = 0)
        row = row + len(dataframe.index) + 2 
xlwriter.save()
dfs

[     csv_st_horizontal  avro_st_horizontal  parquet_st_horizontal  \
 Q1           49.608458           34.927520              24.956528   
 Q2          168.670828           79.977400              57.742420   
 Q3           32.021346           12.211584               6.941551   
 Q4          232.845823          175.171868             169.987462   
 Q5           93.884036           44.462770              33.468086   
 Q6           98.338989           52.109702              47.166969   
 Q7          146.522541           94.447627              74.262720   
 Q8          172.249262          130.898701             131.645367   
 Q9           71.971304           41.511777              33.855555   
 Q10          28.628432           10.192575               4.247113   
 Q11          14.293006            5.348119               1.679916   
 
      orc_st_horizontal  
 Q1           27.277521  
 Q2           58.011053  
 Q3            6.502544  
 Q4          171.147992  
 Q5           32.488883  
 Q

In [26]:
# WRITING THE LOG
import pandas as pd
file_format = ['csv', 'avro', 'parquet', 'orc']
schemas = ['st']
partition = ['horizontal','predicate']

idx = []
li=[]
for i in schemas:
    for j in partition:
        for k in file_format:
            #check folder
            print(f'./logs/{k}/100M_{j}_{i}.txt')

            df = pd.read_csv(f'./logs/{k}/100M_{i}_{j}.txt',sep = ',', header = None)
            df = df.fillna(0)
            avg = df.mean(axis = 0)
            li.append(avg)
            idx.append(k + f"_{i}_{j}")

#create dataframe
df = pd.DataFrame(li, index = [idx])
df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)
# split dataframe
dict = {}
count = 0
loop = 4
dfs = []
for i in range(len(partition)):
    dict['df_{}'.format(i)] = df[count:loop]
    count = loop
    loop = loop+4
    dfs.append(dict[f'df_{i}'])
#######################################################################################
#CREATE RANK OCCURENCES
import scipy.stats as ss
import numpy as np

rank_dataframe = []

for x in dfs:
    df_ranks = x.T
    column_names = df_ranks.columns.to_numpy().tolist()
    
    df_ranks_occurences = []
    for index, row in df_ranks.iterrows():
        df_ranks_occurences.append(ss.rankdata(row))

    df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

    df_transpose = df_ranks_occurences.transpose()

    rank_table = []
    for index, row in df_transpose.iterrows():
        result_row = np.zeros(len(df_transpose.index))
        for i in range(len(row)):
            result_row[int(row[i])-1] +=1
        rank_table.append(result_row)

    rank_table = pd.DataFrame(rank_table)
    rank_table = rank_table.set_axis(column_names, axis = 'index')
    rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')
    rank_dataframe.append(rank_table)
#######################################################################################
# CREATE EXCEL
excel_table = dfs+rank_dataframe
xlwriter = pd.ExcelWriter('~/Desktop/rank_table.xlsx', engine='xlsxwriter')
row = 0
for dataframe in excel_table:
        dataframe.to_excel(xlwriter, sheet_name = 'storage_format_100M', startrow = row, startcol = 0)
        row = row + len(dataframe.index) + 2 
xlwriter.save()

./logs/csv/100M_horizontal_st.txt
./logs/avro/100M_horizontal_st.txt
./logs/parquet/100M_horizontal_st.txt
./logs/orc/100M_horizontal_st.txt
./logs/csv/100M_predicate_st.txt
./logs/avro/100M_predicate_st.txt
./logs/parquet/100M_predicate_st.txt
./logs/orc/100M_predicate_st.txt


In [6]:
# df = pd.DataFrame(li, index = [idx])
# df = df.set_axis(["Q"+str(i+1) for i in range(11)], axis = 1)

# # for x in range(2):
# #     print(x+1)
# #     df+x = df[:4]
# # df1 = df[:4]
# # df2 = df[4:]
# # df2.T

# dict = {}
# count = 0
# loop = 4
# dfs = []
# for i in range(2):
#     dict['df_{}'.format(i)] = df[count:loop].T
#     count = loop
#     loop = loop+4
#     dfs.append(dict[f'df_{i}'])
# dfs[2]
len(partition)
# df1 = pd.DataFrame(li, index=[i + "_st_horizontal" for i in file_format])
# df1 = df1.transpose()
# # # df = df.set_axis(file_format + "_st_horizontal", axis = 1)
# df1 = df1.set_axis(["Q"+str(i+1) for i in range(11)], axis = 0)

2

RANKING TABLE

In [15]:
import scipy.stats as ss
import numpy as np

rank_dataframe = []

for x in dfs:
    df_ranks = x.T
    column_names = df_ranks.columns.to_numpy().tolist()
    
    df_ranks_occurences = []
    for index, row in df_ranks.iterrows():
        df_ranks_occurences.append(ss.rankdata(row))

    df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

    df_transpose = df_ranks_occurences.transpose()

    rank_table = []
    for index, row in df_transpose.iterrows():
        result_row = np.zeros(len(df_transpose.index))
        for i in range(len(row)):
            result_row[int(row[i])-1] +=1
        rank_table.append(result_row)

    rank_table = pd.DataFrame(rank_table)
    rank_table = rank_table.set_axis(column_names, axis = 'index')
    rank_table = rank_table.set_axis([i+1 for i in range(len(column_names))], axis='columns')
    rank_dataframe.append(rank_table)

####################################################################################################################
# df2_ranks = df2
# column_names = df2_ranks.columns.to_numpy().tolist()

# df_ranks_occurences = []
# for index, row in df2_ranks.iterrows():
#     df_ranks_occurences.append(ss.rankdata(row))

# df_ranks_occurences = pd.DataFrame(df_ranks_occurences)

# df_transpose = df_ranks_occurences.transpose()

# df2_ranks = []
# for index, row in df_transpose.iterrows():
#     result_row = np.zeros(len(df_transpose.index))
#     for i in range(len(row)):
#         result_row[int(row[i])-1] +=1
#     df2_ranks.append(result_row)

# df2_ranks = pd.DataFrame(df2_ranks)
# df2_ranks
# df2_ranks = df2_ranks.set_axis(column_names, axis = 'index')
# df2_ranks = df2_ranks.set_axis([i+1 for i in range(len(column_names))], axis='columns')
# print(df2_ranks)

In [23]:
excel_table = dfs + rank_dataframe


In [25]:
len(excel_table)

4